In [1128]:
import numpy as np
from sklearn.covariance import EllipticEnvelope
from sklearn.linear_model import SGDOneClassSVM
import pandas as pd
import ast
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [1129]:
t1Path = 'D:/mt_data/1126_withrw/encoded/t1/'

In [1130]:
device = 'pi4_4G'
tw = 60
feature = 'system calls hashing_1gram'

In [1131]:
filename = 'encoded_bow{}_{}_{}.csv'.format(device, 60, feature)
t1 = t1Path+filename
t1df = pd.read_csv(t1, sep='\t')
t1data = [ast.literal_eval(i) for i in t1df[feature]]
data = pd.DataFrame([t1df['ids'].tolist(), t1df['maltype'].tolist(), t1data]).transpose()

In [1132]:
data.columns = ['ids', 'maltype', feature]

In [1133]:
X = data[data['maltype']=='normal'][feature].tolist()

In [1134]:
y = np.ones(len(X))

In [1135]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.3, shuffle=False) 

In [1136]:
maltypes = ['delay',
 'disorder',
 'freeze',
 'hop',
 'mimic',
 'noise',
 'repeat',
 'spoof']

In [1137]:
X_test=[]
for m in maltypes:
    X_test += data[data['maltype']==m][feature].tolist()

In [1138]:
from sklearn.pipeline import make_pipeline
from sklearn.kernel_approximation import Nystroem
from sklearn.ensemble import IsolationForest

In [1139]:
from sklearn.preprocessing import StandardScaler

In [1140]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)

In [1141]:
pre_train = Nystroem(gamma=0.65, random_state=42, n_components=100)
clf = SGDOneClassSVM(nu=0.05, shuffle=True, fit_intercept=True, random_state=42, tol=1e-6)

In [1142]:
X_train = pre_train.fit_transform(X_train)
X_val = pre_train.transform(X_val)
X_test = pre_train.transform(X_test)

In [1143]:
clf.fit(X_train)

SGDOneClassSVM(nu=0.05, random_state=42, tol=1e-06)

In [1144]:
y_val = [1 for i in range(0, len(X_val))]
y_test = [-1 for i in range(0, len(X_test))]

In [1145]:
metrics.accuracy_score(clf.predict(X_val), y_val)

0.9259259259259259

In [1146]:
metrics.accuracy_score(clf.predict(X_test), y_test)

0.875